 # Transformers Part 1

 ## Table of Contents



 - [Introduction to Transformers](#introduction-to-transformers)

   - [Key Advantages of Transformers](#key-advantages-of-transformers)

 - [1. Attention: The Heart of Transformers](#1-attention-the-heart-of-transformers)

   - [1.1 Transformer Processing](#11-transformer-processing)
   - [1.2 Attention Coefficients](#12-attention-coefficients)
   - [1.3 Self-Attention: Looking at Itself](#13-self-attention-looking-at-itself)
   - [1.4 Adding Learnable Parameters](#14-adding-learnable-parameters)
   - [1.5 Scaled Self-Attention](#15-scaled-self-attention)
     - [Exercise 1: Scaled Dot-Product Self-Attention Dimensions]
   - [1.6 Multi-Head Attention: Multiple Perspectives](#16-multi-head-attention-multiple-perspectives)
     - [Exercise 2: Multi-Head Attention Output]
   - [1.7 Transformer Layers: Putting it Together](#17-transformer-layers-putting-it-together)
     - [Exercise 3: Transformer Layer Components]
   - [1.8 Computational Complexity](#18-computational-complexity)
   - [1.9 Positional Encoding: Where are You?](#19-positional-encoding-where-are-you)
     - [Exercise 4: Purpose of Positional Encoding]

 - [2. Transformers for Natural Language](#2-transformers-for-natural-language)

   - [2.1 Word Embedding: Words as Vectors](#21-word-embedding-words-as-vectors)
     - [Exercise 5: Word Embedding Lookup](#exercise-5-word-embedding-lookup)
   - [2.2 Tokenization: Breaking Down Text](#22-tokenization-breaking-down-text)
   - [2.3 Bag of Words: Simple but Orderless](#23-bag-of-words-simple-but-orderless)
   - [2.4 Autoregressive Models: Adding Order](#24-autoregressive-models-adding-order)
   - [2.5 Recurrent Neural Networks (RNNs): Processing Sequences Step-by-Step](#25-recurrent-neural-networks-rnns-processing-sequences-step-by-step)

 - [Reference](#reference)

 ## Introduction to Transformers



 * **What are they?** Transformers are a groundbreaking development in deep learning. They're neural networks that are particularly good at understanding and processing sequences of data.

 * **Core Idea: Attention!** They use a mechanism called "attention," which lets the network weigh the importance of different parts of the input data when making predictions. Think of it like how you pay more attention to certain words in a sentence to understand its meaning.

 * **Transformation Power:** They are called "transformers" because they transform an input set of data points (vectors) into a new set of data points in a different representation space. The goal is for this new space to have a richer, more useful representation for solving tasks.

 * **Versatility:** Transformers can handle various types of data, like unstructured sets of vectors (e.g., sensor readings), ordered sequences (like text or DNA), and more.

 * **NLP Revolution:** Originally introduced for Natural Language Processing (NLP) tasks like translation, they quickly outperformed older methods like Recurrent Neural Networks (RNNs).

 * **Beyond Text:** They've also shown excellent results in computer vision (image processing), and in models that combine multiple types of data (multimodal transformers for text, images, audio, etc.).



 ---

 ### Key Advantages of Transformers



 * **Transfer Learning:** Models trained on large datasets can be effectively adapted (fine-tuned) for many other specific tasks. These large, adaptable models are known as "foundation models".

 * **Self-Supervised Learning:** They can be trained on vast amounts of unlabeled data (like text from the internet), which is a huge plus.

 * **Scalability (The Scaling Hypothesis):** A key idea is that simply making the models bigger (more parameters) and training them on more data leads to significant performance improvements, even without changing the fundamental architecture.

 * **Parallel Processing:** Transformers are well-suited for modern parallel hardware like GPUs, allowing us to train extremely large models with trillions of parameters. These large models can show surprising "emergent properties"—capabilities that weren't explicitly programmed.



 ---

 ## 1. Attention: The Heart of Transformers



 The core concept that makes transformers so powerful is **attention**. It was first developed as an add-on for RNNs but was later shown to be incredibly effective on its own, leading to transformers replacing RNNs in most applications.



 * **Why Attention? Context Matters!**

     Consider these sentences:

     1.  "I swam across the **river** to get to the other **bank**."

     2.  "I walked across the road to get **cash** from the **bank**."

     The word "bank" means different things. How do we know? By looking at the context words like "river" or "cash". An attention mechanism allows a network to "attend" to these important context words.



     <img src="image/Figure_1.png" width="550px"/>



     *Figure 1: How 'river' and 'swam' influence the meaning of 'bank'. Thicker lines mean more influence.*



 * **Dynamic Weights:** In standard neural networks, the influence of inputs is determined by fixed weights learned during training. In contrast, attention uses weighting factors whose values *depend on the specific input data*.



     <img src="image/Figure_2.jpg" width="550px"/>



     *Figure 2: Example of learned attention weights. Lines show which words attend to others.*



 * **Richer Embeddings:** A transformer can be seen as creating richer "embeddings" (vector representations) where a word's representation depends on the other words in the sequence. So, "bank" in the two example sentences would have different transformed representations.

 * **Example: Proteins:** Proteins are sequences of amino acids. In 3D space, amino acids far apart in the sequence can become physically close and interact. Transformers allow these distant amino acids to "attend" to each other, improving 3D structure modeling.



 ---

 ### 1.1 Transformer Processing



 * **Input = Tokens:** The input to a transformer is a set of vectors \{$x_n$\}, called **tokens**. A token could be a word in a sentence, a patch of an image, or an amino acid in a protein.

 * **Data Matrix X:** These token vectors are usually grouped into a matrix X, where each row is a token.



     <img src="image/Figure_3.png" width="300px"/>



     *Figure 3: The data matrix X, with N tokens and D features per token.*



 * **Transformer Layer:** The basic building block is a function that takes this matrix X and outputs a transformed matrix $\tilde{X}$ of the same size: $\tilde{X} = \text{TransformerLayer}[X]$. Deep networks are built by stacking these layers.

 * **Two Stages in a Layer:**

     1.  **Attention Stage:** Mixes information *across* different tokens.

     2.  **Feed-Forward Stage:** Processes each token *independently*.



 ---

 ### 1.2 Attention Coefficients



 How do we decide which input tokens are important for a given output token?

 * Let's say we want to compute an output vector $y_n$. This $y_n$ will be a weighted sum of all input vectors $x_m$:

     $$y_n = \sum_{m=1}^{N} a_{nm} x_m$$



 * The $a_{nm}$ are the **attention weights** or coefficients.

 * These weights must satisfy two conditions:

     1.  $a_{nm} \ge 0$ (non-negative)

     2.  $\sum_{m=1}^{N} a_{nm} = 1$ (sum to one for each output token $n$)

 * This means attention is distributed – paying more attention to one input means paying less to others. These coefficients are calculated based on the input data itself.



 ---

 ### 1.3 Self-Attention: Looking at Itself



 How are the attention coefficients $a_{nm}$ determined? We use a concept called **self-attention**, drawing an analogy from information retrieval.



 * **Queries, Keys, and Values:**

     * Imagine searching for a movie. You have a **query** (what you're looking for). Each movie in a database has a **key** (its attributes, like genre, actors) and a **value** (the movie file itself). The system matches your query to the keys to find the best movie (value).

 * **In Self-Attention:**

     * Each input token $x_n$ plays three roles:

         1.  It's a **Query (Q)**: Used to ask "who am I similar to?"

         2.  It's a **Key (K)**: Used to say "these are my characteristics."

         3.  It's a **Value (V)**: Used to contribute to the output.

     * Essentially, each token $x_n$ (as a query) is compared against all other tokens $x_m$ (as keys) to determine how much attention $x_n$ should pay to $x_m$.

 * **Calculating Similarity:** A simple way to measure similarity between a query $x_n$ and a key $x_m$ is their **dot product**: $x_n^T x_m$.

 * **Softmax for Weights:** To satisfy the non-negativity and sum-to-one constraints for attention weights $a_{nm}$, we use the softmax function on these dot products:

     $$a_{nm} = \frac{\exp(x_n^T x_m)}{\sum_{m'=1}^{N} \exp(x_n^T x_{m'})}$$



 * **Output Calculation:** The output $y_n$ is then a weighted sum of the *value* vectors (which, in basic self-attention, are just the input vectors $x_m$ themselves).

 * **Matrix Form:** $$Y = \text{Softmax}[XX^T]X$$ (where X provides queries, keys, and values). This is called **dot-product self-attention**.



 ---

 ### 1.4 Adding Learnable Parameters



 The basic self-attention has no learnable parameters. To make it learnable and more flexible:



 * We introduce separate learnable weight matrices $W^{(q)}$, $W^{(k)}$, and $W^{(v)}$ to transform the input X into Queries (Q), Keys (K), and Values (V) respectively:

     * $Q = XW^{(q)}$

     * $K = XW^{(k)}$

     * $V = XW^{(v)}$

 * These W matrices allow the network to learn how to best project the input tokens into representations suitable for querying, keying, and valuing.

 * The dimensions of Q and K must allow dot products. Typically, the output V has the same dimension as the input X to facilitate stacking layers and residual connections.

 * The attention output is then: $$Y = \text{Softmax}[QK^T]V$$



     <img src="image/Figure_4.png" width="550px"/>



     *Figure 4: Calculating $QK^T$ for attention coefficients.*



     <img src="image/Figure_5.png" width="550px"/>



     *Figure 5: Calculating the final output Y using $QK^T$ and V.*



 ---

 ### 1.5 Scaled Self-Attention



 * **The Problem:** If the dot products $QK^T$ become very large, the gradients of the softmax function can become tiny, making learning slow or ineffective.

 * **The Solution:** Scale the dot products before the softmax. If query and key vectors have elements with zero mean and unit variance, their dot product has a variance of $D_k$ (the dimension of keys). So, we scale by $1/\sqrt{D_k}$.

 * **Scaled Dot-Product Self-Attention**:

     $$Y = \text{Attention}(Q, K, V) = \text{Softmax}\left(\frac{QK^T}{\sqrt{D_k}}\right)V$$



     <img src="image/Figure_6.png" width="250px"/>



     *Figure 6: Information flow in a scaled dot-product self-attention head.*



 ---

 #### Algorithm 1: Scaled Dot-Product Self-Attention (PyTorch Implementation)



 Here's a PyTorch implementation of the Scaled Dot-Product Self-Attention mechanism.

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class ScaledDotProductAttention(nn.Module):
    """
    Computes Scaled Dot-Product Attention.
    Input:
        Q (Query): Tensor of shape (batch_size, n_queries, d_k)
        K (Key):   Tensor of shape (batch_size, n_keys, d_k)
        V (Value): Tensor of shape (batch_size, n_keys, d_v)
        mask (optional): Tensor for masking future tokens (batch_size, n_queries, n_keys)
    Output:
        context: Tensor of shape (batch_size, n_queries, d_v)
        attn_weights: Tensor of shape (batch_size, n_queries, n_keys)
    """
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, mask=None):
        d_k = Q.size(-1)  # Dimension of keys (and queries)

        # 1. Compute QK^T
        # Q: (batch_size, n_queries, d_k)
        # K.transpose(-2, -1): (batch_size, d_k, n_keys)
        # scores: (batch_size, n_queries, n_keys)
        scores = torch.matmul(Q, K.transpose(-2, -1))

        # 2. Scale by sqrt(d_k)
        scaled_scores = scores / math.sqrt(d_k)

        # 3. Apply mask (if provided)
        # The mask sets future positions to -infinity before softmax
        if mask is not None:
            # Ensure mask has the same shape or is broadcastable
            # Mask values are typically 0 for positions to attend to, and 1 for masked positions.
            # We want to set masked positions to a very small number (-inf) before softmax.
            scaled_scores = scaled_scores.masked_fill(mask == 1, -1e9) # or float('-inf')

        # 4. Apply softmax to get attention weights
        # attn_weights: (batch_size, n_queries, n_keys)
        attn_weights = F.softmax(scaled_scores, dim=-1)

        # 5. Multiply by V to get context vectors
        # attn_weights: (batch_size, n_queries, n_keys)
        # V: (batch_size, n_keys, d_v)
        # context: (batch_size, n_queries, d_v)
        context = torch.matmul(attn_weights, V)

        return context, attn_weights

# Example Usage:
# Let's define some dimensions
batch_size = 2
n_queries = 5  # Sequence length of queries (e.g., target sequence)
n_keys = 7     # Sequence length of keys/values (e.g., source sequence)
d_k = 64       # Dimension of keys/queries
d_v = 128      # Dimension of values

# Create dummy tensors
Q = torch.randn(batch_size, n_queries, d_k)
K = torch.randn(batch_size, n_keys, d_k)
V = torch.randn(batch_size, n_keys, d_v)

# Optional: Create a mask (e.g., for causal attention in decoders)
# This mask would prevent attention to future tokens.
# For self-attention where n_queries == n_keys, a causal mask would be upper triangular.
# Here, n_queries != n_keys, so a causal mask isn't directly applicable in the same way.
# If this were self-attention (n_queries=n_keys=seq_len), a causal mask:
# seq_len = 5
# causal_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
# causal_mask = causal_mask.unsqueeze(0).expand(batch_size, -1, -1) # (batch_size, seq_len, seq_len)

# Instantiate the attention module
attention_module = ScaledDotProductAttention()

# Get the output
context_vectors, attention_weights = attention_module(Q, K, V) #, mask=causal_mask if applicable)

print("Shape of Q:", Q.shape)
print("Shape of K:", K.shape)
print("Shape of V:", V.shape)
print("Shape of Context Vectors:", context_vectors.shape)
print("Shape of Attention Weights:", attention_weights.shape)
# print("Attention Weights (first batch, first query):\n", attention_weights[0, 0, :])


Shape of Q: torch.Size([2, 5, 64])
Shape of K: torch.Size([2, 7, 64])
Shape of V: torch.Size([2, 7, 128])
Shape of Context Vectors: torch.Size([2, 5, 128])
Shape of Attention Weights: torch.Size([2, 5, 7])


 #### <a id="exercise-1-scaled-dot-product-self-attention-dimensions"></a>Exercise 1: Scaled Dot-Product Self-Attention Dimensions



 **Question:** In the Scaled Dot-Product Self-Attention, if your Query matrix `Q` has shape `(batch_size, 10, 32)`, your Key matrix `K` has shape `(batch_size, 12, 32)`, and your Value matrix `V` has shape `(batch_size, 12, 64)`, what will be the shape of the output context vector and the attention weights?

 #### <a id="solution-1"></a>Solution 1

`solve here`

 ---

 ### 1.6 Multi-Head Attention: Multiple Perspectives



 A single attention mechanism ("head") might average out different types of relationships (e.g., grammatical vs. semantic).



 * **Solution:** Use multiple attention heads in parallel, each with its own set of $W^{(q)}$, $W^{(k)}$, and $W^{(v)}$ matrices. This is like using multiple filters in a Convolutional Neural Network (CNN) layer.

 * For H heads, we get H output matrices $H_1, ..., H_H$.

 * These are concatenated and then linearly transformed by another weight matrix $W^{(o)}$ to produce the final output Y, which has the same dimension as the input X:

     $$Y(X) = \text{Concat}[H_1, ..., H_H]W^{(o)}$$



     <img src="image/Figure_7.png" width="450px"/>



     *Figure 7: Multi-head attention architecture. Outputs of heads are combined.*



     <img src="image/Figure_8.png" width="500px"/>



     *Figure 8: Information flow for multi-head attention.*



 * Typically, if the input/output dimension is D (embed_dim), each head might output a $D_v = D/H$ dimensional vector, so the concatenated vector is D-dimensional. The key/query dimension per head is often $D_k = D/H$ as well.



 ---

 #### Algorithm 2: Multi-Head Attention (PyTorch Implementation from scratch)



 This implementation shows how Multi-Head Attention can be built using multiple Scaled Dot-Product Attention heads.

In [8]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention module.
    Input:
        X (Input Tensor): Tensor of shape (batch_size, seq_len, embed_dim)
        mask (optional): Mask for attention.
    Output:
        output: Tensor of shape (batch_size, seq_len, embed_dim)
        attn_weights: Tensor of shape (batch_size, num_heads, seq_len, seq_len) (or seq_len_q, seq_len_k)
    """
    def __init__(self, embed_dim, num_heads,_bias=True): # Added _bias for compatibility with nn.MultiheadAttention
        super(MultiHeadAttention, self).__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"

        self.embed_dim = embed_dim  # Total dimension of the model (D)
        self.num_heads = num_heads  # Number of attention heads (H)
        self.head_dim = embed_dim // num_heads  # Dimension of each head (d_k = d_v = D/H)

        # Linear layers for Q, K, V for all heads (can be done in one go)
        # These will project X from embed_dim to embed_dim (num_heads * head_dim)
        # W_q, W_k, W_v in the paper are effectively these linear layers.
        # Each W_h^(q), W_h^(k), W_h^(v) processes X to get Q_h, K_h, V_h.
        # We can implement this by having one large linear layer and then splitting.
        self.W_q = nn.Linear(embed_dim, embed_dim, bias=_bias) # Projects X to Q' (embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim, bias=_bias) # Projects X to K' (embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim, bias=_bias) # Projects X to V' (embed_dim)

        # Scaled dot-product attention module (one for all heads after splitting)
        self.attention = ScaledDotProductAttention()

        # Final linear layer W_o
        self.W_o = nn.Linear(embed_dim, embed_dim, bias=_bias)

    def split_heads(self, x, batch_size):
        """
        Splits the last dimension into (num_heads, head_dim).
        Input x: (batch_size, seq_len, embed_dim)
        Output: (batch_size, num_heads, seq_len, head_dim)
        """
        x = x.view(batch_size, -1, self.num_heads, self.head_dim)
        return x.transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)

    def combine_heads(self, x, batch_size):
        """
        Combines the heads back from (batch_size, num_heads, seq_len, head_dim)
        to (batch_size, seq_len, embed_dim).
        """
        x = x.transpose(1, 2).contiguous() # (batch_size, seq_len, num_heads, head_dim)
        return x.view(batch_size, -1, self.embed_dim) # (batch_size, seq_len, embed_dim)

    def forward(self, X_query, X_key, X_value, mask=None):
        # X_query, X_key, X_value are expected to be (batch_size, seq_len, embed_dim)
        # For self-attention, X_query = X_key = X_value = X
        batch_size = X_query.size(0)

        # 1. Linearly project Q, K, V for all heads
        # Q_proj, K_proj, V_proj will have shape (batch_size, seq_len, embed_dim)
        Q_proj = self.W_q(X_query)
        K_proj = self.W_k(X_key)
        V_proj = self.W_v(X_value)

        # 2. Split into multiple heads
        # Q_heads, K_heads, V_heads will have shape (batch_size, num_heads, seq_len, head_dim)
        Q_heads = self.split_heads(Q_proj, batch_size)
        K_heads = self.split_heads(K_proj, batch_size)
        V_heads = self.split_heads(V_proj, batch_size)

        # 3. Apply scaled dot-product attention for each head
        # context_heads: (batch_size, num_heads, seq_len_q, head_dim)
        # attn_weights_heads: (batch_size, num_heads, seq_len_q, seq_len_k)
        # If mask is provided, it should be compatible with (batch_size, num_heads, seq_len_q, seq_len_k)
        # or broadcastable, e.g., (batch_size, 1, seq_len_q, seq_len_k)
        if mask is not None:
            mask = mask.unsqueeze(1) # Add head dimension for broadcasting

        context_heads, attn_weights_heads = self.attention(Q_heads, K_heads, V_heads, mask)

        # 4. Concatenate heads (combine_heads)
        # context_concat: (batch_size, seq_len_q, embed_dim)
        context_concat = self.combine_heads(context_heads, batch_size)

        # 5. Final linear projection (W_o)
        # output: (batch_size, seq_len_q, embed_dim)
        output = self.W_o(context_concat)

        return output, attn_weights_heads


# Example Usage (Self-Attention):
embed_dim = 256
num_heads = 8
batch_size = 4
seq_len = 10

# Input tensor X
X = torch.randn(batch_size, seq_len, embed_dim)

# Instantiate MultiHeadAttention
mha = MultiHeadAttention(embed_dim, num_heads)

# Forward pass (for self-attention, query, key, and value are the same)
output, attention_weights = mha(X, X, X)

print("Shape of Input X:", X.shape)
print("Shape of Output from MultiHeadAttention:", output.shape)
print("Shape of Attention Weights from MultiHeadAttention:", attention_weights.shape)
# attention_weights[0,0] would be the attention map for the first head of the first batch item.


Shape of Input X: torch.Size([4, 10, 256])
Shape of Output from MultiHeadAttention: torch.Size([4, 10, 256])
Shape of Attention Weights from MultiHeadAttention: torch.Size([4, 8, 10, 10])


 #### Using `torch.nn.MultiheadAttention`



 PyTorch provides a built-in `MultiheadAttention` layer which is optimized and handles much of the complexity internally.



 **Key parameters for `torch.nn.MultiheadAttention`:**

 - `embed_dim`: Total dimension of the model.

 - `num_heads`: Number of parallel attention heads. `embed_dim` will be split across `num_heads`.

 - `dropout`: Dropout probability on attn_output_weights. Default: 0.0.

 - `bias`: If `True`, add bias to input/output projections. Default: `True`.

 - `batch_first`: If `True`, then the input and output tensors are provided as (batch, seq, feature). Default: `False` (seq, batch, feature). **It's common to set this to `True`**.

In [9]:
# Example using torch.nn.MultiheadAttention
embed_dim_pt = 256
num_heads_pt = 8
batch_size_pt = 4
seq_len_pt = 10 # For query
seq_len_kv_pt = 12 # For key/value, can be different for cross-attention

# Input tensors
query = torch.randn(batch_size_pt, seq_len_pt, embed_dim_pt) # (N, L, E) if batch_first=True
key   = torch.randn(batch_size_pt, seq_len_kv_pt, embed_dim_pt) # (N, S, E) if batch_first=True
value = torch.randn(batch_size_pt, seq_len_kv_pt, embed_dim_pt) # (N, S, E) if batch_first=True

# Instantiate nn.MultiheadAttention
# IMPORTANT: Set batch_first=True if your inputs are (batch, seq, feature)
pytorch_mha = nn.MultiheadAttention(embed_dim=embed_dim_pt, num_heads=num_heads_pt, batch_first=True)

# Forward pass
# The nn.MultiheadAttention layer expects query, key, value.
# For self-attention, query, key, and value are the same.
# It returns:
# attn_output: (N, L, E) if batch_first=True
# attn_output_weights: (N, L, S) - average weights over heads if need_weights=True
# (N, num_heads, L, S) if average_attn_weights=False (PyTorch 1.9+)
# For simplicity, we'll use the default average_attn_weights=True
attn_output, attn_output_weights = pytorch_mha(query, key, value) # No mask applied here for simplicity

print("Using torch.nn.MultiheadAttention:")
print("Shape of Query:", query.shape)
print("Shape of Key:", key.shape)
print("Shape of Value:", value.shape)
print("Shape of Attention Output:", attn_output.shape)
print("Shape of Attention Output Weights (averaged):", attn_output_weights.shape)

# Self-attention example with nn.MultiheadAttention
X_pt = torch.randn(batch_size_pt, seq_len_pt, embed_dim_pt)
self_attn_output, self_attn_weights = pytorch_mha(X_pt, X_pt, X_pt)
print("\nSelf-Attention with nn.MultiheadAttention:")
print("Shape of X_pt:", X_pt.shape)
print("Shape of Self-Attention Output:", self_attn_output.shape)
print("Shape of Self-Attention Weights (averaged):", self_attn_weights.shape)


# To get per-head attention weights (PyTorch 1.9+):
# pytorch_mha_per_head = nn.MultiheadAttention(embed_dim=embed_dim_pt, num_heads=num_heads_pt, batch_first=True, average_attn_weights=False)
# _, attn_output_weights_per_head = pytorch_mha_per_head(query, key, value, need_weights=True)
# print("Shape of Attention Output Weights (per head):", attn_output_weights_per_head.shape) # (N, num_heads, L, S)



Using torch.nn.MultiheadAttention:
Shape of Query: torch.Size([4, 10, 256])
Shape of Key: torch.Size([4, 12, 256])
Shape of Value: torch.Size([4, 12, 256])
Shape of Attention Output: torch.Size([4, 10, 256])
Shape of Attention Output Weights (averaged): torch.Size([4, 10, 12])

Self-Attention with nn.MultiheadAttention:
Shape of X_pt: torch.Size([4, 10, 256])
Shape of Self-Attention Output: torch.Size([4, 10, 256])
Shape of Self-Attention Weights (averaged): torch.Size([4, 10, 10])


 #### <a id="exercise-2-multi-head-attention-output"></a>Exercise 2: Multi-Head Attention Output



 **Question:** If you have an input `X` of shape `(batch_size=16, seq_len=20, embed_dim=512)` and you use a Multi-Head Attention layer with `num_heads=8`, what is the expected shape of the final output of this layer? What is the dimension (`head_dim`) of queries, keys, and values within each attention head?

 #### <a id="solution-2"></a>Solution 2

`solve here`



 ---

 ### 1.7 Transformer Layers: Putting it Together



 A full transformer layer combines multi-head attention with other components:



 1.  **Multi-Head Self-Attention:** As described above.

 2.  **Add & Norm (Residual Connection + Layer Normalization):**

     * The output of the multi-head attention is added to the original input X (a **residual connection**). This helps with training deep networks.

     * Then, **Layer Normalization** is applied, which stabilizes training.

     * So, $$Z = \text{LayerNorm}[\text{MultiHeadAttention}(X) + X]$$

 3.  **Feed-Forward Network (Position-wise MLP):**

     * The attention mechanism is primarily linear in how it combines value vectors. To add more non-linearity and processing power, the output Z from the "Add & Norm" step is passed through a standard Multi-Layer Perceptron (MLP).

     * This MLP is applied independently to each token's representation (i.e., to each row of Z). It typically has one hidden layer.

 4.  **Add & Norm (Again):**

     * Another residual connection and layer normalization:

         $$\tilde{X} = \text{LayerNorm}[\text{MLP}(Z) + Z]$$



 This entire structure forms one **transformer layer**. Multiple such layers are stacked to build deep transformer models.



 <img src="image/Figure_9.png" width="250px"/>



 *Figure 9: Architecture of one transformer layer.*



 ---

#### Algorithm 3: Transformer Layer (Conceptual PyTorch Implementation)



 This shows how a single Transformer Encoder layer is constructed. `torch.nn.TransformerEncoderLayer` provides this out-of-the-box.

In [10]:
class PositionwiseFeedForward(nn.Module):
    """ Implements the Position-wise FeedForward network (MLP). """
    def __init__(self, embed_dim, ffn_dim, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(embed_dim, ffn_dim)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(ffn_dim, embed_dim)
        # ReLU is commonly used as the activation function
        self.activation = nn.ReLU()

    def forward(self, x):
        # x: (batch_size, seq_len, embed_dim)
        x = self.linear1(x)         # (batch_size, seq_len, ffn_dim)
        x = self.activation(x)      # (batch_size, seq_len, ffn_dim)
        x = self.dropout(x)         # (batch_size, seq_len, ffn_dim)
        x = self.linear2(x)         # (batch_size, seq_len, embed_dim)
        return x

class TransformerEncoderLayerScratch(nn.Module):
    """
    A single Transformer Encoder Layer implemented from scratch components.
    """
    def __init__(self, embed_dim, num_heads, ffn_dim, dropout=0.1):
        super(TransformerEncoderLayerScratch, self).__init__()
        # Multi-Head Self-Attention
        self.self_attn = MultiHeadAttention(embed_dim, num_heads) # Using our scratch MHA
        self.norm1 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout)

        # Position-wise Feed-Forward Network
        self.ffn = PositionwiseFeedForward(embed_dim, ffn_dim, dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):
        # src: (batch_size, seq_len, embed_dim)
        # src_mask: for self-attention, e.g., padding mask

        # 1. Multi-Head Self-Attention part
        # attn_output: (batch_size, seq_len, embed_dim)
        attn_output, _ = self.self_attn(src, src, src, mask=src_mask)
        # Add & Norm (Residual connection + LayerNorm)
        # src has shape (batch_size, seq_len, embed_dim)
        # attn_output has shape (batch_size, seq_len, embed_dim)
        out1 = self.norm1(src + self.dropout1(attn_output))

        # 2. Feed-Forward part
        # ffn_output: (batch_size, seq_len, embed_dim)
        ffn_output = self.ffn(out1)
        # Add & Norm
        # out1 has shape (batch_size, seq_len, embed_dim)
        # ffn_output has shape (batch_size, seq_len, embed_dim)
        out2 = self.norm2(out1 + self.dropout2(ffn_output))

        return out2

# Example Usage:
embed_dim_layer = 256
num_heads_layer = 8
ffn_dim_layer = 512 # Typically 2x to 4x embed_dim
batch_size_layer = 4
seq_len_layer = 10
dropout_layer = 0.1

src_tensor = torch.randn(batch_size_layer, seq_len_layer, embed_dim_layer)

# Using the scratch implementation
encoder_layer_scratch = TransformerEncoderLayerScratch(embed_dim_layer, num_heads_layer, ffn_dim_layer, dropout_layer)
output_scratch = encoder_layer_scratch(src_tensor)
print("Shape of Input to TransformerEncoderLayer (Scratch):", src_tensor.shape)
print("Shape of Output from TransformerEncoderLayer (Scratch):", output_scratch.shape)


Shape of Input to TransformerEncoderLayer (Scratch): torch.Size([4, 10, 256])
Shape of Output from TransformerEncoderLayer (Scratch): torch.Size([4, 10, 256])


 #### Using `torch.nn.TransformerEncoderLayer`



 PyTorch provides a convenient `TransformerEncoderLayer` module.



 **Key parameters for `torch.nn.TransformerEncoderLayer`:**

 - `d_model`: The number of expected features in the input (i.e., `embed_dim`).

 - `nhead`: The number of heads in the multiheadattention models.

 - `dim_feedforward`: The dimension of the feedforward network model. Default: 2048.

 - `dropout`: The dropout value. Default: 0.1.

 - `activation`: The activation function of the intermediate layer, can be a string ("relu" or "gelu") or a unary callable. Default: “relu”.

 - `batch_first`: If `True`, then the input and output tensors are provided as (batch, seq, feature). Default: `False`. **Set this to `True` for consistency.**

 - `norm_first`: If `True`, layer norm is applied before attention and feedforward operations, otherwise after. Default: `False`. (Pre-LN can sometimes lead to more stable training).

In [11]:
# Example using torch.nn.TransformerEncoderLayer
pytorch_encoder_layer = nn.TransformerEncoderLayer(
    d_model=embed_dim_layer,
    nhead=num_heads_layer,
    dim_feedforward=ffn_dim_layer,
    dropout=dropout_layer,
    batch_first=True # Important!
)

output_pytorch_layer = pytorch_encoder_layer(src_tensor) # src_mask can also be passed
print("\nShape of Output from torch.nn.TransformerEncoderLayer:", output_pytorch_layer.shape)

# Note: To build a full Transformer Encoder, you would stack multiple TransformerEncoderLayer
# instances using torch.nn.TransformerEncoder.
# transformer_encoder = nn.TransformerEncoder(pytorch_encoder_layer, num_layers=6)
# output_full_encoder = transformer_encoder(src_tensor)
# print("Shape of Output from torch.nn.TransformerEncoder (6 layers):", output_full_encoder.shape)



Shape of Output from torch.nn.TransformerEncoderLayer: torch.Size([4, 10, 256])


 #### <a id="exercise-3-transformer-layer-components"></a>Exercise 3: Transformer Layer Components



 **Question:** What are the two main sub-layers within a standard Transformer Encoder layer, and what is typically applied after each of these sub-layers?

 #### <a id="solution-3"></a>Solution 3

`solve here`

 ---

 ### 1.8 Computational Complexity



 * A transformer layer is generally more computationally efficient than a fully connected network for sequence data.

 * The attention part has a computational cost roughly proportional to $N^2D$ (where N is sequence length, D is token dimension).

 * The MLP part costs about $ND^2$.

 * The number of parameters is mainly in $D^2$.



 ---

 ### 1.9 Positional Encoding: Where are You?



 * **The Problem:** The self-attention mechanism itself doesn't care about the order of tokens. If you shuffle the input tokens, the output tokens will be the same but also shuffled (this is called permutation equivariance). But for many tasks, like language, order is crucial! "Man bites dog" is different from "Dog bites man".

 * **The Solution:** We need to give the model information about the position of each token in the sequence. This is done by adding a **positional encoding vector** $r_n$ to each input token embedding $x_n$:

     $$\tilde{x}_n = x_n + r_n$$



 * The positional encoding vector $r_n$ has the same dimension as the token embedding $x_n$.

 * **Sinusoidal Positional Encodings:** A common method uses sine and cosine functions of different frequencies for different dimensions of the encoding vector:

     For a position `pos` and dimension `i` (where `i` ranges from 0 to `embed_dim-1`):

     $$

     PE_{(pos, 2i)} = \sin(pos / 10000^{2i/embed\_dim})

     $$

     $$

     PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/embed\_dim})

     $$

     (The $10000$ is a hyperparameter $L$ from the formula; $embed\_dim$ is $D$).

     * This creates unique encodings for each position, is bounded, can generalize to longer sequences, and makes it easier for the model to learn about relative positions.



     <img src="image/Figure_10_a.png" width="350px"/>



     *(a) Visualization of sinusoidal functions for positional encoding components.*



     <img src="image/Figure_10_b.png" width="350px"/>



     *(b) Heatmap of positional encoding vectors.*



 * **Learned Positional Encodings:** Another approach is to learn the positional encoding vectors during training (e.g., using an `nn.Embedding` layer where the input is position indices). This works well if sequence lengths are relatively constant but may not generalize to unseen lengths as effectively as sinusoidal encodings.



 ---

 #### PyTorch Implementation of Sinusoidal Positional Encoding

In [12]:
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000, dropout=0.1):
        """
        Args:
            embed_dim (int): Dimension of the model embedding.
            max_len (int): Maximum possible sequence length.
            dropout (float): Dropout rate.
        """
        super(SinusoidalPositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Create a long enough P matrix that can be sliced
        # P: (max_len, embed_dim)
        position = torch.arange(max_len).unsqueeze(1) # (max_len, 1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim)) # (embed_dim/2)

        pe = torch.zeros(max_len, embed_dim)
        pe[:, 0::2] = torch.sin(position * div_term) # Apply to even indices
        pe[:, 1::2] = torch.cos(position * div_term) # Apply to odd indices

        # pe is of shape (max_len, embed_dim)
        # We want to register it as a buffer so it's part of the model's state,
        # but not trained as a parameter.
        # We add an unsqueezed dimension for batch compatibility (though it's often sliced).
        self.register_buffer('pe', pe.unsqueeze(0)) # (1, max_len, embed_dim)

    def forward(self, x):
        """
        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_len, embed_dim)
                        or (seq_len, batch_size, embed_dim) if not batch_first.
                        We assume batch_first=True for this example based on common use.
        Returns:
            Tensor: Output tensor with positional encodings added.
                    Shape: (batch_size, seq_len, embed_dim)
        """
        # x is (batch_size, seq_len, embed_dim)
        # self.pe is (1, max_len, embed_dim)
        # We need to add pe[:, :x.size(1)] to x
        # The positional encoding is added to the input embeddings.
        # The `pe` buffer is sliced up to the sequence length of the input `x`.
        # x.size(1) gives the sequence length.
        # self.pe[:, :x.size(1), :] has shape (1, seq_len, embed_dim)
        # This will broadcast across the batch dimension of x.
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

# Example Usage:
embed_dim_pe = 512
max_seq_len_pe = 100
batch_size_pe = 4
current_seq_len_pe = 20 # Actual sequence length of the current batch

# Create dummy input embeddings (e.g., after word embedding layer)
input_embeddings = torch.randn(batch_size_pe, current_seq_len_pe, embed_dim_pe)

# Instantiate positional encoding
pos_encoder = SinusoidalPositionalEncoding(embed_dim=embed_dim_pe, max_len=max_seq_len_pe)

# Add positional encodings
output_with_pe = pos_encoder(input_embeddings)

print("Shape of Input Embeddings:", input_embeddings.shape)
print("Shape of Output with Positional Encodings:", output_with_pe.shape)
# Check if values changed (they should have)
# print("Original first token embedding (sum):", input_embeddings[0,0,:].sum())
# print("First token embedding with PE (sum):", output_with_pe[0,0,:].sum())


Shape of Input Embeddings: torch.Size([4, 20, 512])
Shape of Output with Positional Encodings: torch.Size([4, 20, 512])


 #### <a id="exercise-4-purpose-of-positional-encoding"></a>Exercise 4: Purpose of Positional Encoding



 **Question:** Why is positional encoding necessary in Transformer models, especially when processing sequential data like text? What would happen if it were omitted?

 #### <a id="solution-4"></a>Solution 4


`solve here`

 ---

 ## 2. Transformers for Natural Language



 Transformers were first developed for language tasks and have since become state-of-the-art.





 ### 2.1 Word Embedding: Words as Vectors



 * **The Challenge:** Computers need numbers, not words. How do we represent words numerically?

 * **One-Hot Encoding:** Assign each word a unique vector with one '1' and the rest '0's. Simple, but:

     * Vectors become very long for large vocabularies (e.g., hundreds of thousands of dimensions).

     * Doesn't capture similarity (e.g., "cat" and "kitten" are just as different as "cat" and "car").

 * **Word Embeddings (Dense Vectors):** Map words into a lower-dimensional space (e.g., a few hundred dimensions) where similar words have similar vector representations.

     * This is done using an embedding matrix E. If $x_n$ is a one-hot vector for a word, its embedding is $v_n = Ex_n$. In practice, we use integer indices for words and an embedding layer looks up these indices.

 * **Learning Embeddings (e.g., word2vec):**

     * Embeddings are learned from large amounts of text. A popular method is **word2vec**.

     * It's essentially a simple neural network trained on a "predict the context" or "predict the word from context" task.

         * **Continuous Bag of Words (CBOW):** Predict the middle word from its surrounding context words.

         * **Skip-grams:** Predict the surrounding context words given the middle word.



         <img src="image/Figure_11_a.png" width="350px"/>



         *(a) Continuous Bag of Words (CBOW)*



         <img src="image/Figure_11_b.png" width="350px"/>



         *(b) Skip-grams*

     * This self-supervised learning helps capture semantic relationships. For example, vector("Paris") - vector("France") + vector("Italy") ≈ vector("Rome").

     * Word embeddings can be pre-trained or learned as the first layer of a larger deep learning model.



 ---

 #### PyTorch `nn.Embedding` Layer



 PyTorch provides `torch.nn.Embedding` which is a simple lookup table that stores embeddings of a fixed dictionary and size.

In [13]:
# Example of nn.Embedding
num_embeddings = 1000  # Vocabulary size (K)
embedding_dim = 300   # Dimension of the embedding space (D)

# Create an embedding layer
embedding_layer = nn.Embedding(num_embeddings, embedding_dim)

# Example input: a batch of sequences of word indices
# Let's say batch_size = 2, sequence_length = 5
# Word indices should be between 0 and num_embeddings-1
input_indices = torch.LongTensor([[10, 2, 45, 700, 3],
                                  [200, 34, 5, 0, 999]]) # Shape: (2, 5)

# Get the embeddings
output_embeddings = embedding_layer(input_indices) # Shape: (2, 5, 300)

print("Shape of Input Indices:", input_indices.shape)
print("Shape of Output Embeddings:", output_embeddings.shape)
# print("Embedding for the first word of the first sequence ('10'):\n", output_embeddings[0, 0, :])

# The weights of the embedding layer (the embedding matrix E) can be accessed:
# print("Shape of Embedding Layer Weights:", embedding_layer.weight.shape) # (num_embeddings, embedding_dim)


Shape of Input Indices: torch.Size([2, 5])
Shape of Output Embeddings: torch.Size([2, 5, 300])


 #### <a id="exercise-5-word-embedding-lookup"></a>Exercise 5: Word Embedding Lookup



 **Question:** You have an `nn.Embedding` layer initialized with `num_embeddings=5000` (vocabulary size) and `embedding_dim=100`. If you provide an input tensor of word indices with shape `(batch_size=4, sequence_length=15)`, what will be the shape of the output tensor from this embedding layer?

 #### <a id="solution-5"></a>Solution 5

`solve here`

 ---

 ### 2.2 Tokenization: Breaking Down Text



 * **Beyond Fixed Dictionaries:** Using a fixed dictionary of words has problems with:

     * Words not in the dictionary (Out-Of-Vocabulary or OOV words).

     * Misspellings, punctuation, or other character sequences like code.

 * **Character-Level:** Using individual characters as input solves OOV but loses word structure and makes sequences very long.

 * **Tokenization:** The common solution is to break text into **tokens**, which are often sub-word units (small groups of characters).

     * This allows the model to handle OOV words by representing them as a sequence of known sub-word tokens.

     * Common words might be single tokens, while rare words are broken down.

     * Helps with word variations like "cook," "cooks," "cooked," "cooking" – they share the "cook" token.

 * **Byte Pair Encoding (BPE):** A popular tokenization algorithm.

     1.  Start with individual characters as the initial set of tokens.

     2.  Iteratively find the most frequently occurring adjacent pair of tokens in the text and merge them to form a new token.

     3.  Repeat until a desired vocabulary size is reached.



     <img src="image/Figure_12.png" width="450px"/>



     *Figure 12: Example of Byte Pair Encoding. 'pe' is merged first, then 'ck', etc.*



 ---

 ### 2.3 Bag of Words: Simple but Orderless



 A very basic way to model a sequence of words $x_1, ..., x_N$:

 * Assume all words are drawn independently from the same distribution:

     $$p(x_1, ..., x_N) = \prod_{n=1}^N p(x_n)$$



 * This model completely **ignores the order of words** (hence "bag of words").

 * Useful for simple tasks like **text classification** (e.g., sentiment analysis using Naive Bayes, where word probabilities differ per class).



 ---

 ### 2.4 Autoregressive Models: Adding Order



 To account for word order, we can use autoregressive models:

 * The joint probability of a sequence is factored as a product of conditional probabilities:

     $$p(x_1, ..., x_N) = \prod_{n=1}^N p(x_n | x_1, ..., x_{n-1})$$



     (The probability of a word depends on all preceding words).

 * **n-gram Models:** A simplification where the probability of a word depends only on the $L$ preceding words (e.g., $L=1$ for bi-grams, $L=2$ for tri-grams).

     * For $L=2$ (tri-gram):

         $$p(x_1, ..., x_N) = p(x_1)p(x_2|x_1)\prod_{n=3}^N p(x_n | x_{n-1}, x_{n-2})$$



 * **Limitation:** These models struggle to capture **long-range dependencies** in language because the context is limited, and the number of parameters grows exponentially with $L$.



 ---

 ### 2.5 Recurrent Neural Networks (RNNs): Processing Sequences Step-by-Step



 RNNs were the standard for sequence modeling before transformers.

 * **Core Idea:** Process a sequence one element at a time, maintaining a **hidden state** $z_n$ that summarizes the information seen so far.

 * The network takes the current input $x_n$ and the previous hidden state $z_{n-1}$ to produce an output $y_n$ and the next hidden state $z_n$. Weights are shared across time steps.



     <img src="image/Figure_13.png" width="450px"/>



     *Figure 13: A general Recurrent Neural Network (RNN).*



 * **Example: Machine Translation (Encoder-Decoder RNN)**

     * **Encoder:** An RNN reads the input sentence (e.g., English) and compresses its meaning into a final hidden state vector $z^*$.

     * **Decoder:** Another RNN takes $z^*$ and generates the output sentence (e.g., Dutch) word by word.



     <img src="image/Figure_14.png" width="550px"/>



     *Figure 14: RNN for language translation (encoder-decoder architecture).*



 * **Training RNNs:** Using an algorithm called Backpropagation Through Time (BPTT).

 * **Problems with RNNs:**

     1.  **Vanishing/Exploding Gradients:** During BPTT for long sequences, gradients can become extremely small (vanish) or large (explode), making training difficult.

     2.  **Long-Range Dependencies:** Standard RNNs struggle to capture relationships between distant elements in a sequence.

     3.  **Information Bottleneck:** In encoder-decoder models, the entire input sequence's meaning must be squeezed into a single fixed-size vector $z^*$, which is a major limitation for long sequences.

     4.  **Sequential Computation:** They process sequences step-by-step, which limits parallelization on modern hardware like GPUs.

 * **LSTMs and GRUs:** More advanced RNN variants (Long Short-Term Memory and Gated Recurrent Units) were developed to mitigate these issues but still have limitations with very long dependencies and are often slower to train.

 * **Transformers were designed to overcome these RNN limitations**.



 ---

 ---

 ### <a id="reference"></a>Reference



 Bishop, C. M. (2024). *Deep Learning: Foundations and Concepts*. Springer. (Chapter 12: Transformers).